# Airbnb price prediction modeling competition hosted by SIGNATE

* The competition is already terminated and is no longer accessible on the site
* The submission with the following code was ranked 8th / 931 participants with RMSE around 140.9

### 1. Preparation : word extraction for the scoring of natural language columns

##### 1-1. Extract words from the description column and the name column, and assign the median value to each word

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [ ]:
# Load the training data from a CSV file, using the first column as the index
df=pd.read_csv("train.csv", index_col=0)

# Create a new column 'descname' by concatenating 'description' and 'name' columns with a space separator
df["descname"] = df["description"] + " " + df["name"]

In [ ]:
# Initialize CountVectorizer to convert text to a binary matrix representation
# binary=True means only presence/absence of a word is counted, ignoring multiple occurrences
# stop_words='english' removes common English stop words
vectorizer = CountVectorizer(stop_words='english', binary=True)

# Fit the vectorizer on the 'descname' column and transform the text data into a sparse matrix
X = vectorizer.fit_transform(df['descname'])

# Calculate the total count of each unique word across all documents
word_counts = X.toarray().sum(axis=0)

# Get the list of unique words (features) extracted by the vectorizer
word_list = vectorizer.get_feature_names_out()

# Create a DataFrame with two columns: 'word' and its corresponding 'count'
word_frequency = pd.DataFrame({'word': word_list, 'count': word_counts})


In [ ]:
# Get the price values array
y_values = df['y'].values

# Compute median price for each word
median_prices = []
for i, word in enumerate(word_list):
    rows_with_word = X[:, i].toarray().flatten()  # rows that have a certain word
    if rows_with_word.sum() > 0:  
        relevant_y_values = y_values[rows_with_word == 1]  
        median_price = np.median(relevant_y_values) 
    else:
        median_price = np.nan  
    median_prices.append(median_price)

# Add the median prices in the DataFrame
word_frequency['median_price'] = median_prices

# Filter the words by the count >= 10
filtered_word_frequency = word_frequency[word_frequency['count'] >= 10]

# Export the DataFrame to .xlsx
filtered_word_frequency.sort_values(by='count', ascending=False).to_excel('Wordlist.xlsx', index=False)

##### 1-2. Decompose amenity column strings into words
* *After exporting the dataframe, I gave a point to each amenity item manually on a scale of 0 to 4 by how luxurious the facility I thought it would imply.*<br>
* *However, if it is too cumbersome, it is also possible to use the median value by item, with the same method as the description/name columns*

In [ ]:
import re
from collections import defaultdict

# Initialize a default dictionary to count the occurrence of each amenity
dic = defaultdict(int)

# Loop through each row in the 'amenities' column
for _ in range(df["amenities"].shape[0]):

  # Extract the string inside curly braces, e.g., '{Wifi, TV}' -> 'Wifi, TV'
  keys = re.findall(r'\{(.*)\}', df.loc[_,"amenities"])[0].split(",")
  
  # Split the string by commas and count each amenity
  for key in keys:
    dic[key] += 1

# Convert the dictionary into a DataFrame
amenities_df = pd.DataFrame(list(dic.items()), columns=['amenity', 'count'])

# Export the DataFrame to an Excel file
amenities_df.to_excel('amenity_scores.xlsx', index=False)

### 2. Import necessary data

In [ ]:
# Load the raw training data and set the first column as the index
df=pd.read_csv("train.csv", index_col=0)

# Load the amenity score and wordlist data from an Excel file
amenity_scores_df = pd.read_excel("amenity_scores.xlsx")
word_scores_df = pd.read_excel("Wordlist.xlsx")

# Convert the scoring data to dictionary
# For amenity, Key=item and value=score, and for wordlist, key = word and value = score 
amenity_scores_dict = pd.Series(amenity_scores_df.score.values, index=amenity_scores_df.word).to_dict()
word_scores_dict = pd.Series(word_scores_df.score.values, index=word_scores_df.word).to_dict()

### 3. Data Preprocessinng

##### 3-1. Preprocess the training data

In [ ]:
def data_pre(df):
  
  # ====== Missing values imputation ======

  # Fill missing values in bathroom/bedroom/bed columns by median value grouped by df['accommodates']
  for column in ['bathrooms', 'bedrooms', 'beds']:
      if df[column].isnull().sum() > 0:
          medians = df.groupby('accommodates')[column].median()

          df[column] = df.apply(
              lambda row: medians[row['accommodates']] if pd.isnull(row[column]) else row[column], axis=1
          )
  # Fill missing rating with overall median
  df['review_scores_rating'] = df['review_scores_rating'].fillna(df['review_scores_rating'].median())

  # Fill missing categorical host data with 'f' (false)
  df['host_identity_verified'] = df['host_identity_verified'].fillna('f')
  df["host_has_profile_pic"] = df["host_has_profile_pic"].fillna('f')

  # Clean and fill missing 'host_response_rate'
  df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype('float')
  df['host_response_rate'] = df['host_response_rate'].fillna(df['host_response_rate'].median())
  
  # Convert availability of thumbnail URL into binary (1 if exists, 0 otherwise)
  df['thumbnail_url'] = df['thumbnail_url'].notna().astype(int)


  # ====== Handle datetime columns ======

  # Convert 'host_since' to datetime and fill missing with median
  df['host_since'] = pd.to_datetime(df['host_since'])
  df['host_since']=df['host_since'].fillna(df['host_since'].median())

  # Convert 'first_review' and 'last_review' to datetime
  df["first_review"] = pd.to_datetime(df["first_review"])
  df['first_review']=df['first_review'].fillna(df['host_since'])

  df["last_review"] = pd.to_datetime(df["last_review"])
  df['last_review']=df['last_review'].fillna(df['host_since'])

  # Convert datetime to numeric values (days or UNIX timestamp)
  df['host_days_since'] = (pd.Timestamp('2017-10-05') - df['host_since']).dt.days
  df["first_review"] = df["first_review"].astype('int64') // 10**9
  df["last_review"] = df["last_review"].astype('int64') // 10**9


  # ====== Feature engineering ======

  # Clean the amenities string
  df['cleaned_amenities'] = df['amenities'].str.replace('"', '', regex=False)

  # Calculate the amenity score from predefined dictionary
  def calculate_amenity_score(cleaned_amenities):
      total_score = 0
      keys = re.findall(r'\{(.*)\}', cleaned_amenities)
      if keys:
          for key in keys[0].split(","):
              key = key.strip()
              total_score += amenity_scores_dict.get(key, 0)
      return total_score
  
  df['amenity_scores'] = df['cleaned_amenities'].apply(calculate_amenity_score)

  # Calculate the name score from predefined dictionary
  def calculate_name_score(name):
      total_score = 0
      words = name.split() 
      for word in words:
          word = word.strip().lower()  
          if word in word_scores_dict:
              total_score += word_scores_dict[word] 
      return total_score
  df['name_scores'] = df['name'].apply(calculate_name_score)

  # Calculate the description score from predefined dictionary
  def calculate_description_score(description):
      total_score = 0
      words = description.split()
      for word in words:
          word = word.strip().lower()
          if word in word_scores_dict:
              total_score += word_scores_dict[word]
      return total_score
  df['description_scores'] = df['description'].apply(calculate_description_score)

  # Count number of words in the 'description' field
  df["description_wordcount"] = df["description"].apply(lambda x: len(str(x).split()))


 # ====== Drop unused columns ======

  drop_list=["amenities","city","description","neighbourhood","name",'cleaned_amenities',"host_since","zipcode"]
  df=df.drop(drop_list,axis=1)


  # Convert object-type columns to category dtype
  object_cols = df.select_dtypes(include='object').columns
  df[object_cols] = df[object_cols].astype('category')

  return df

In [ ]:
# Apply the preprocessing function
df=data_pre(df)

In [ ]:
# ====== Target encoding the training data ======

# Get the categorical columns
categorical_columns = df.select_dtypes(include=['category', 'object']).columns

# Prepare the empty dictionary to store target encoding mappings for each categorical column
target_encodings = {}

# Apply target encoding on training data
for col in categorical_columns:
    # Calculate the mean of the target valuable for each category
    mean_prices = df.groupby(col)["y"].mean()

    # Create and store the mapping dictionary
    target_encoding = {category: mean for category, mean in mean_prices.items()}
    target_encodings[col] = target_encoding  # 保存

    # Map categories to their mean target value and convert to float
    df[f"{col}_encoded"] = df[col].map(target_encoding).astype(float)

# Drop original categorical columns
df = df.drop(columns=categorical_columns)

# Display the result
print(df.head())


##### 3-2. Preprocess the test data

In [ ]:
# Load the training data again and apply the predefined function
# This DataFrame will be used for the target encoding of the test data
df2 = pd.read_csv("train.csv", index_col=0)
df2 = data_pre(df2)

In [ ]:
#Load and apply the predefined function to the test data
df_test=pd.read_csv("test.csv", index_col=0)
df_test=data_pre(df_test)

In [ ]:
# ====== Target encoding the test data ======

# Identify categorical columns in the test data
test_categorical_columns = df_test.select_dtypes(include=['category', 'object']).columns

# Replace unseen categories in test data with 'Other'
for col in test_categorical_columns:
    # Get category values from training data
    train_categories = df2[col].unique()

    # Replace categories in test data not seen in training data with 'Other'
    df_test[col] = df_test[col].apply(
        lambda x: x if x in train_categories else 'Other'
    )

# Empty dictionary to store target encoding mappings for each categorical column
test_target_encodings = {}

# Apply target encoding to test data using training data statistics
for col in test_categorical_columns:
    # Calculate the mean of the taret variable for each category in training data
    mean_prices = df2.groupby(col)["y"].mean()

    # Create and store the mapping dictionary
    target_encoding = {category: mean for category, mean in mean_prices.items()}
    target_encoding["Other"]=df2["y"].mean() #Use global mean for unseen categories
    test_target_encodings[col] = target_encoding  

    # Map categories to their mean target value and convert to float
    df_test[f"{col}_encoded"] = df_test[col].map(target_encoding).astype(float)

# Drop the original categorical columns
drop_list_test = df_test.select_dtypes(include=['category',"object"]).columns
df_test = df_test.drop(columns=drop_list_test)

# Display the result
print(df_test.head())

### 4. Data Modeling using LightGBM

In [ ]:
import optuna.integration.lightgbm as lgb_tune
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Split the dataset into training and validation sets
df_train, df_val = train_test_split(df, test_size=0.2)

# Define target and feature columns for training data
col = "y"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

# Define target and feature columns for validation data
val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

# Create LightGBM dataset objects
trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

# Set basic parameters for LightGBM with Optuna tuning
params = {
    "objective": "regression",
    "metric": "rmse",
}

# Train the model with Optuna-tuned LightGBM
model_tune = lgb_tune.train(
    params, trains, valid_sets=[valids],
    callbacks=[
        lgb.early_stopping(100),  
        lgb.log_evaluation(100)   
    ]
)

# Predict on the validation set and calculate RMSE
val_preds = model_tune.predict(val_x)
val_rmse = mean_squared_error(val_y, val_preds, squared=False)
print(f'Validation RMSE: {val_rmse}')

In [ ]:
# Load and preprocess the test dataset
df_test=pd.read_csv("test.csv", index_col=0)
df_test=data_pre(df_test)

# Predict on the test dataset
predict = model_tune.predict(df_test)

### 5. Postprocessing and export of the submission file

In [ ]:
# Replace predicted prices lower than 10 with 10 to handle unrealistic prediction
predict = [max(10, pred) for pred in predict]

# Assign the adjusted predictions back to the test DataFrame
df_test["y"] = predict

# Export the predictions as a CSV file without header
df_test["y"].to_csv("submission.csv", header=False)